In [1]:
#USE PYTHON 3.10

from db_utils import *
from config import generate_config_json
from ask_tell_utils import descriptive_ask, csv_tell

import dragonfly #DO NOT REINSTALL--THE SITE-PACKAGES VERSION OF DRAGONFLY IS FIXED/MODIFIED
from dragonfly import load_config_file
from dragonfly.exd.experiment_caller import CPFunctionCaller
from dragonfly.opt import gp_bandit

/home/ubuntu/.local/lib/python3.10/site-packages/dragonfly/utils/oper_utils.py:30: UserWarning: cannot import name 'direct' from 'dragonfly.utils.direct_fortran' (/home/ubuntu/.local/lib/python3.10/site-packages/dragonfly/utils/direct_fortran/__init__.py)
Could not import Fortran direct library. Dragonfly can still be used, but might be slightly slower. To get rid of this warning, install a numpy compatible Fortran compiler (e.g. gfortran) and the python-dev package and reinstall Dragonfly.
  warn('%s\n%s'%(e, fortran_err_msg))


## **INPUT PARAMETERS**

*Example Usage:*
> ALL_CONSIDERED_COMPONENTS = [
> 
>     ('Ca(ClO4)2', (0,7.5)),
> 
>     ('LiPF6', (0,7.5)),
> 
>     ('CaCl2', (0,7.5)),
> 
>     ('H6C4O3',(0,7.5))
> 
> ]
> 
> EXP_COMPONENTS = [
> 
>     'CaCl2',
> 
>     'H6C4O3',
> 
>     'LiPF6'
> 
> ]
> 
> MAX_VOL = 100.0
> 
> MIN_VOL = 2.0
>
> CELL_CONSTANT = 10
> 
> EXPERIMENTER_INITIALS = 'AC'
> 
> CONFIG_OUTPUT_FILE = './config'
>
> CSV_FILEPATH = '/home/ubuntu/eis_db/acquire/data/eis_circle_fit_test_data.csv'

In [3]:
ALL_CONSIDERED_COMPONENTS = [
    ('Ca(ClO4)2', (0,7.5)), #Perchlorate
    ('H8C4O2S', (0,5.0)), #Sulfolane
    ('H7C3NO',(0,5.0)) #DMF
]
EXP_COMPONENTS = [
    'Ca(ClO4)2',
    'H8C4O2S',
    'H7C3NO'
]
# in mL
MAX_VOL = 6.0
MIN_VOL = 5.0
CELL_CONSTANT = 10

EXPERIMENTER_INITIALS = 'AC'
CONFIG_OUTPUT_FILE = './config'
CSV_FILEPATH = '/home/ubuntu/eis_db/acquire/data/eis_circle_fit_test_data.csv'

## **EXPERIMENT SETUP**

**Click play below to generate new config file**

In [4]:

#GENERATE CONFIG FILES
all_considered_components_formatted = []
for chem, bounds in ALL_CONSIDERED_COMPONENTS:
    all_considered_components_formatted.append((get_component_type(chem),bounds))

exp_components_formatted = [get_component_type(chem) for chem in EXP_COMPONENTS]
exp_config_path, constraint_path = generate_config_json(
    all_considered_components_formatted,
    exp_components_formatted,
    MAX_VOL,
    MIN_VOL,
    EXPERIMENTER_INITIALS,
    CONFIG_OUTPUT_FILE
                                       )
print(exp_config_path + ': New config file generated')

No components found for formula H3C7NO
No components found for formula H3C7NO


AttributeError: 'NoneType' object has no attribute 'is_salt'

**Click play below to create optimizer and read in space config file.**

To read old config file, replace *exp_config_path* in:
> config = load_config_file(exp_config_path)

*This will take a minute or so*

In [4]:
#INPUTS CONFIG FILE INTO OPTIMIZER

config = load_config_file(exp_config_path)
#optimization method is bayesian optimization
domain, domain_orderings = config.domain, config.domain_orderings
func_caller = CPFunctionCaller(None, domain, domain_orderings=domain_orderings)
opt = gp_bandit.CPGPBandit(func_caller, ask_tell_mode=True)
opt.initialise()

/home/ubuntu/.local/lib/python3.10/site-packages/dragonfly/exd/exd_core.py:338: UserWarning: Sampling an initial pool failed despite 10 attempts -- will continue trying but consider reparametrising your domain if this problem persists.
  warn(('Sampling an initial pool failed despite %d attempts -- will ' +
/home/ubuntu/.local/lib/python3.10/site-packages/dragonfly/exd/exd_core.py:338: UserWarning: Sampling an initial pool failed despite 20 attempts -- will continue trying but consider reparametrising your domain if this problem persists.
  warn(('Sampling an initial pool failed despite %d attempts -- will ' +
/home/ubuntu/.local/lib/python3.10/site-packages/dragonfly/exd/exd_core.py:338: UserWarning: Sampling an initial pool failed despite 30 attempts -- will continue trying but consider reparametrising your domain if this problem persists.
  warn(('Sampling an initial pool failed despite %d attempts -- will ' +
/home/ubuntu/.local/lib/python3.10/site-packages/dragonfly/exd/exd_core.p

**Click play below to get all relevant datapoints from database to start off with.**

In [5]:
#GETS ALL RELEVANT ELECTROLYTE DATA FROM DATABASE; INPUTS INTO OPTIMIZER

experiment_electrolytes = get_experiment_electrolytes([tupl[0] for tupl in ALL_CONSIDERED_COMPONENTS])
opt.tell([(point['amounts'], point['electrolyte'][1]) for point in experiment_electrolytes])
#TELL ALL EXISTING DATABASE POINTS

## **IN-EXPERIMENT USE**

### ***ASK:***
**Click play below to ask for a new point**

The given point does not necessarily have to be evaluated.

In [6]:

current_electrolyte = descriptive_ask(opt, ALL_CONSIDERED_COMPONENTS)
print(current_electrolyte)

{'Ca(ClO4)2': 0.0, 'LiPF6': 1.355, 'CaCl2': 3.649, 'H6C4O3': 3.882}

### ***TELL:***

**Click play below to input a new experimental result**

*Do not use this to tell the optimizer a given experiment was impossible*

It is important to specify an x_cutoff for the circle fit to begin ignoring *for each inputted point*--typically this is used as the x value where the warburg element begins.

In [ ]:
X_CUTOFF = 65.85

csv_tell(current_electrolyte, opt, CSV_FILEPATH, X_CUTOFF, CELL_CONSTANT)

### ***TELL*** *(Impossible Experiment)*:

**Click play below to tell the asked point was not evaluable, because of solubility reasons or otherwise.**

In [ ]:
opt.tell([(current_electrolyte.values(), -1)])